# Fossasia 2017, Simmi Mourya

See github.com/simmimourya1/fossasia_17

In [ ]:
%load_ext cython

In [ ]:
import sys
import Cython
print("Python %d.%d.%d %s %s" % sys.version_info)
print("Cython %s" % Cython.__version__)

# Functions and Coercion

In [ ]:
%%cython

def pyfunc(x):
    return x + 1

def cyfunc(int x):
    return x + 1

cdef int cfunc(int x):
    return x + 1

cpdef cypyfunc(int x):
    y = cfunc(x + 1)
    return y * 2

In [ ]:
pyfunc(2)

In [ ]:
cyfunc(2)

In [ ]:
cfunc(2)

In [ ]:
cypyfunc(2)

# Static typing and type inference

In [ ]:
import math

def sin(x):
    return math.sin(x)

In [ ]:
%%cython -a
cimport libc.math

def sin(double x):
    return libc.math.sin(x)

In [ ]:
%%cython -a
def local_variables(x):
    cdef int i = 5, ix = x
    print(i * ix)
    return (i + ix) // 2

In [ ]:
local_variables(2)

## Python vs Cython

In [ ]:
import math

def py_circular_distance(radius, lon1, lat1, lon2, lat2):
    x = math.pi/180.0
    a = (90.0-lat1) * x
    b = (90.0-lat2) * x
    theta = (lon2-lon1) * x
    c = math.acos((math.cos(a)*math.cos(b)) + (math.sin(a)*math.sin(b)*math.cos(theta)))
    return radius*c


In [ ]:
%%cython -a
import math

def cy_circular_distance(radius, lon1, lat1, lon2, lat2):
    x = math.pi/180.0
    a = (90.0-lat1) * x
    b = (90.0-lat2) * x
    theta = (lon2-lon1) * x
    c = math.acos((math.cos(a)*math.cos(b)) + (math.sin(a)*math.sin(b)*math.cos(theta)))
    return radius*c


In [ ]:
print(py_circular_distance(10, 1.2, 2, 2, 4.3))
print(cy_circular_distance(10, 1.2, 2, 2, 4.3))

In [ ]:
%timeit py_circular_distance(10, 1.2, 2, 2, 4.3)

In [ ]:
%timeit cy_circular_distance(10, 1.2, 2, 2, 4.3)

# Calling C functions

In [ ]:
%%cython -a
# libc math functions
from libc cimport math

print( math.sin(math.M_PI / 2) )


## Cython and Numpy

In [ ]:
%%cython
import numpy as np

def qm_cython_first_pass(double x0, int n):
    cdef int t
    x = np.zeros(n+1, float)
    x[0] = x0
    for t in range(n):
        x[t+1] = 4.0 * x[t] * (1 - x[t])
    return np.asarray(x)

*The problem of computing iterates and returning a time series requires us to work with arrays
The natural array type to work with is NumPy arrays
Here’s a Cython implemention that initializes, populates and returns a NumPy array*

In [ ]:
timeit qm_cython_first_pass(0.1, int(10**5))

The reason is that working with NumPy arrays incurs substantial Python overheads. 

We can do better by using Cython’s typed memoryviews, which provide more direct access to arrays in memory.

When using them, the first step is to create a NumPy array.

Next, we declare a memoryview and bind it to the NumPy array.

Here’s an example:

In [ ]:
%%cython
import numpy as np
from numpy cimport float_t

def qm_cython(double x0, int n):
    cdef int t
    x_np_array = np.zeros(n+1, dtype=float)
    cdef float_t [:] x = x_np_array
    x[0] = x0
    for t in range(n):
        x[t+1] = 4.0 * x[t] * (1 - x[t])
    return np.asarray(x)

In [ ]:
timeit qm_cython(0.1, int(10**5))

Here

cimport pulls in some compile-time information from NumPy

cdef float_t [:] x = x_np_array creates a memoryview on the NumPy array x_np_array

the return statement uses np.asarray(x) to convert the memoryview back to a NumPy array

In [ ]:
%%cython
from cython.view cimport array as cvarray
import numpy as np

# Memoryview on a NumPy array
narr = np.arange(27, dtype=np.dtype("i")).reshape((3, 3, 3))
cdef int [:, :, :] narr_view = narr

# Memoryview on a C array
cdef int carr[3][3][3]
cdef int [:, :, :] carr_view = carr

# Memoryview on a Cython array
cyarr = cvarray(shape=(3, 3, 3), itemsize=sizeof(int), format="i")
cdef int [:, :, :] cyarr_view = cyarr

# Show the sum of all the arrays before altering it
print("NumPy sum of the NumPy array before assignments: %s" % narr.sum())

# We can copy the values from one memoryview into another using a single
# statement, by either indexing with ... or (NumPy-style) with a colon.
carr_view[...] = narr_view
cyarr_view[:] = narr_view
# NumPy-style syntax for assigning a single value to all elements.
narr_view[:, :, :] = 3

# Just to distinguish the arrays
carr_view[0, 0, 0] = 100
cyarr_view[0, 0, 0] = 1000

# Assigning into the memoryview on the NumPy array alters the latter
print("NumPy sum of NumPy array after assignments: %s" % narr.sum())
